# Pipeline

### Load audio
Puede ser mp3 o wav, puede ser mono o estereo. 
### Convert into normalize Array 
Vector de que representa para un periodo de tiempo la intensidad para cada frecuencia (frecuencia determina la dimensión del vector, si es más de un canal, tendrá 2 vectores cada periodo de tiempo). El periodo puede ser 1 seg o 20 Milisegundos (normalmente son de 20 milisegundos, como lo es este ejemplo). Si
un vector tiene 16.000 Hz como frecuencia máxima, entonces tendra dim=16.000
### Convert to spectogram (creo que en este ejemplo no se hace)
Lo convierte en una imagen, el cual es lo que consume el modelo. Creo que se usa fast fourier transform. Despues torch lo transforma en un tensor.
### Speech Model
Ya con el input, aca estará el modelo que se entrenará para predecir fonemas para cada periodo de tiempo. Wav2Vec2 (ENCODER) es uno. Lo bueno de transformers, es que predice considerando el todo, no de forma independiente a los demás ponemas.
Aca se podría hacer un finetuning con esto: https://huggingface.co/blog/fine-tune-xlsr-wav2vec2

El output será 1 vector por cada 20 milisegundos, con la probabilidad de cada fonema (la dimensión va a depender de la cantidad de fonemas/vocabulario con el que contemos. Los labels). A eso se le puede hacer un argumax para extraer el mas probable, o correrle algo como beam search para que traiga las combinaciones mas probables considerando un modelo linguistico.

### CTC language model (Beam Search)
Aca es donde tengo que ivnestigar más. En wev2vec puede que este considerado ya. Lo que si hay que investigar, es beam search,
que lo que hace es identificar las palabras más comunes en su totalidad

In [ ]:
# Instalar aplicaciónes de nose q cosa linux: https://apps.microsoft.com/store/detail/windows-subsystem-for-linux-preview/9P9TQF7MRM4R?hl=es-ad&gl=ad&icid=CatNavWindowsApps

# info de decoder:
# https://huggingface.co/blog/wav2vec2-with-ngram
# https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Boosting_Wav2Vec2_with_n_grams_in_Transformers.ipynb#scrollTo=aJREUIdqs5ak

In [ ]:
# urls interesantes

Aca puede explicar como aplica CTC a wav2vec2: https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Boosting_Wav2Vec2_with_n_grams_in_Transformers.ipynb#scrollTo=dAerOhydrNFR

In [ ]:
#Generar dataset con audios propios

Carga audio y lo convierte a array y nose si a spectogram tambien

In [9]:
# pip install audio2numpy
import audio2numpy as a2n


#Read the audio data from the file and load it into a 2D Numpy array. (can be .wav o .mp3)
x,sr=a2n.audio_from_file("wqwq.wav")
# En este ejemplo ("wqwq.wav"), x, tiene una dimensión de 751.616, y el audio es de 46seg. Por lo que se cumple que, 751.616/16.000=46. cada segundo tiene un vector de 16.000. En cada segundo, hay una foto de 16.000 frecuencias con el valor de la intesnidad.

#This array consists of a sequence of numbers, each representing a measurement of the intensity or amplitude of the sound at a particular moment in time. The number of such measurements is determined by the sampling rate. For instance, if the sampling rate was 44.1kHz, the Numpy array will have a single row of 44,100 numbers for 1 second of audio.
# Audio can have one or two channels, known as mono or stereo, in common parlance. With two-channel audio, we would have another similar sequence of amplitude numbers for the second channel. In other words, our Numpy array will be 3D, with a depth of 2.

Dataset=[[dict({'array':common_voice[0],'path':f'audio{1}','sampling_rate':common_voice[1]})],[dict({'array':common_voice[0],'path':f'audio{1}','sampling_rate':common_voice[1]})]]
label=['hola lo estamos llamando','hola lo  estamos para aca']

In [ ]:
# Convert to uniform dimensions: sample rate, channels, and duration

We might have a lot of variation in our audio data items. Clips might be sampled at different rates, or have a different number of channels. The clips will most likely have different durations. As explained above this means that the dimensions of each audio item will be different.
Since our deep learning models expect all our input items to have a similar size, we now perform some data cleaning steps to standardize the dimensions of our audio data. We resample the audio so that every item has the same sampling rate. We convert all items to the same number of channels. All items also have to be converted to the same audio duration. This involves padding the shorter sequences or truncating the longer sequences.
If the quality of the audio was poor, we might enhance it by applying a noise-removal algorithm to eliminate background noise so that we can focus on the spoken audio.

In [ ]:
# Data Augmentation of raw audio

# We could apply some data augmentation techniques to add more variety to our input data and help the model learn to generalize to a wider range of inputs. We could Time Shift our audio left or right randomly by a small percentage, or change the Pitch or the Speed of the audio by a small amount.





# Mel Spectrograms

# This raw audio is now converted to Mel Spectrograms. A Spectrogram captures the nature of the audio as an image by decomposing it into the set of frequencies that are included in it.


# MFCC

# For human speech, in particular, it sometimes helps to take one additional step and convert the Mel Spectrogram into MFCC (Mel Frequency Cepstral Coefficients). MFCCs produce a compressed representation of the Mel Spectrogram by extracting only the most essential frequency coefficients, which correspond to the frequency ranges at which humans speak.




# So our model takes the Spectrogram images and outputs character probabilities for each timestep or ‘frame’ in that Spectrogram.

PROCESADOR

In [6]:
# PROCESSOR.Se pueden crear o descargar de un preentrenado. Es quien guarda el vocabulario, dimensiones de entrada y salida, padding, mask, y ese tipo de cosas

# 3 formas  (feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor):
#        -processors como  Wav2Vec2Processor (tokenizer + feature extractor) o Wav2Vec2ProcessorWithLM (tokenizer + feature extractor+ decoder)
#        -tokenizer + feature extractor
#        - También se le puede definir un decoder ()


from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2FeatureExtractor

#Opción 1, la mas simple
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
# Wav2Vec2Processor:Constructs a Wav2Vec2 processor which wraps a Wav2Vec2 feature extractor and a Wav2Vec2 CTC tokenizer into a single processor.
# should be used for padding and conversion into a tensor of type torch.



#Opción 2, toekizer e featureExtractor

#Define el label y su configuración
tokenizer=Wav2Vec2CTCTokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

#Define el output e input y las configruaciones
FeatureExtracor=Wav2Vec2FeatureExtractor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

tokenizer.get_vocab() #Para ver el vocabulario o modificarlo (son los posibles labels)
" ".join(sorted(processor.tokenizer.get_vocab()))  #Para ver el vocabulario o modificarlo (son los posibles labels)
len(tokenizer.get_vocab()) #ver dim salida, o qty de labels posibles


#Opción 3

# pip install transformers==4.18.0
# pip install pyctcdecode
# pip install https://github.com/kpu/kenlm/archive/master.zip 

from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
# feature_extractor (Wav2Vec2FeatureExtractor) — An instance of Wav2Vec2FeatureExtractor. The feature extractor is a required input.
# tokenizer (Wav2Vec2CTCTokenizer) — An instance of Wav2Vec2CTCTokenizer. The tokenizer is a required input.
# decoder (pyctcdecode.BeamSearchDecoderCTC) — An instance of pyctcdecode.BeamSearchDecoderCTC. The decoder is a required input.


MODELO

In [67]:
# MODELS (https://huggingface.co/docs/transformers/model_doc/wav2vec2)
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
# Wav2Vec2ForCTC: se carga el modelo
# AutoModelForCTC
# Wav2Vec2Model
# Wav2Vec2ForCTC
# Wav2Vec2ForSequenceClassification
# Wav2Vec2ForAudioFrameClassification
# Wav2Vec2ForXVector
# Wav2Vec2ForPreTraining

predict

In [10]:
#Convierte el input en un tensor+attention mask. prepara para enviarlo al modelo
inputs = processor(x, sampling_rate=16_000, return_tensors="pt")
# Ajusta formato (dejan de ser float, y son 2 tensor: inputs y mask)

In [11]:
import torch

with torch.no_grad():
  logits = model(**inputs).logits

#El modelo procesa el input y genera para cada 20 milisegundos un vector con 41 valores. 
# En este caso hay un total de 2349 períodos, donde cada uno es un tensor de dimensión 41 (el vocab. labels posibles) con probabilidades

C:\Users\Usuario\anaconda3\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [12]:
#hago el argmax. Para cada uno de los 2.349 periodos de tiempo en los 46 seg, se selecciona el fonema con mayor probabilidad
predicted_ids = torch.argmax(logits, dim=-1)


# Con logits, es donde puedo aplicar beam search


#Según los fonemas maximos para cada periodo de tiempo, con batch_decode identifico los labels y aplico el diccionario
transcription = processor.batch_decode(predicted_ids)

transcription[0].lower()

'aquí me aparece once cerotres cincuenta y cuatrolistsi gas natural propano aquí me aparece es consulta de base de datos once diecinueve noventa y cuatro y se hace con la referenciacon referencia si señol pero entonces es que así está cargado a nivel nacional ningún punto de servicio lo carga e con el código de barras'

In [ ]:
# Como saber como subdividir/segmentar el audio para que prediga los fonemas? cada fonema puede durar distinto, hay pausas, algunos van juntos, apple como sabe que son 2 p?
#Aca aparece CTC Algorithm (Connectionist Temporal Classification) — Training and Inference,

# CTC Loss (during Training):
# It has a ground truth target transcript and tries to train the network to maximize the probability of outputting that correct transcript.

# CTC Decoding (during Inference): 
# Here we don’t have a target transcript to refer to, and have to predict the most likely sequence of characters.
# Use the character probabilities to pick the most likely character for each frame, including blanks. eg. “-G-o-ood”
# 2 opciones:
#     Greedy Search: Para cada periodo, simplemente elige el fonema de mayor probabilidad
#    (Language Model) Beam Search: Para cada preiodo busca los fonemas que en relaciones a los que lo rodean, tienen mayor probabilidad de generar una palabra o frase. ej: HAC tiene mayor probabilidad que JAK en español

In [ ]:
# Medición y evaluación con Metrics — Word Error Rate (WER)